In [1]:
pip install python-chess

     -------------------------------------- 154.4/154.4 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [130]:
import chess
import random
import numpy as np
#def main():
random.seed(123)

def sim_game(nsim):
    result = np.zeros(nsim)
    for i in range(nsim):
        board = chess.Board()
        while not board.is_game_over():
            legal_moves = list(board.legal_moves)
            #print("Legal moves:", [move.uci() for move in legal_moves])
        
            #user_move = input("Enter your move (e.g., 'e2e4'): ")
        
            try:
                ## User-generated move
                #move = chess.Move.from_uci(user_move)
                if board.legal_moves.count() == 0:
                    #print(board.result)
                    break
                ## Random user move
                #user_move = random.choice(list(board.legal_moves))
                ## Engine move
                user_move = next_move(board)
                #print(user_move)
                board.push(user_move)

                #if move in legal_moves:
                    #board.push(move)
                    #print(board)
                
                ## Make a random move for the computer
                if board.legal_moves.count() == 0:
                    #print(board.result)
                    break
                computer_move = random.choice(list(board.legal_moves))
                board.push(computer_move)
                #print("Computer's move:", computer_move.uci())
                #print(board)
                #else:
                #    print("Invalid move. Try again.")
            except ValueError:
                print("Invalid move format. Try again.")
        if board.result().partition('-')[0] == '1/2':
            result[i] = 0.5
        else:
            result[i] = board.result().partition('-')[0]
        if (i+1) % 10 == 0:
            print(f"Game {i+1} Result: {board.result()}")
    return np.mean(result)
    
sim_game(100)
#if __name__ == "__main__":
#    main()

Game 10 Result: 1/2-1/2
Game 20 Result: 1/2-1/2
Game 30 Result: 1/2-1/2
Game 40 Result: 1/2-1/2
Game 50 Result: 1/2-1/2
Game 60 Result: 1/2-1/2
Game 70 Result: 1/2-1/2
Game 80 Result: 1-0
Game 90 Result: 1-0
Game 100 Result: 1/2-1/2


0.635

In [43]:
import chess
board = chess.Board()
chess.LegalMoveGenerator(board)

<LegalMoveGenerator at 0x15d3d7e4460 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>

In [104]:
board.result()

'*'

In [35]:
chess.square_knight_distance(chess.Square(2), chess.Square(9))

2

In [42]:
import chess.engine
print(chess.engine.Score.score(board))

None


In [90]:
piece_values = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 10
    # No value for KING, as its loss results in the end of the game
}

def score_board(board):
    score = 0
    
    # Iterate through all squares
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            if piece.color == chess.WHITE:
                score += piece_values[piece.piece_type]
            else:
                score -= piece_values[piece.piece_type]
    
    return score


def next_move(board, moves = 1):
    moves = board.legal_moves
    scores = []
    for i in board.legal_moves:
        board.push(i)
        scores.append(score_board(board))
        ## Undo move
        board.pop()
    max_value = np.max(scores)
    max_indices = np.where(scores == max_value)[0]
    return list(moves)[random.choice(max_indices)]
#    return list(moves)[scores.index(max(scores))]

        
#next_move(board)

In [91]:
next_move(board)

Move.from_uci('e2e4')

1

In [129]:
np.log(0.6165 / (1 - 0.6165)) * 400 - 100

89.88748071883535